In [ ]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from scipy.stats.mstats import winsorize
from pmdarima import auto_arima
from statsmodels.tsa.holtwinters import Holt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from scipy.stats import pearsonr, spearmanr, kendalltau

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df1 = pd.read_csv('EXPCH.csv',parse_dates=True,index_col='DATE')
df2 = pd.read_csv('IMPCH.csv',parse_dates=True,index_col='DATE')
print(df1)
print(df2)

In [ ]:
df1.shape

In [ ]:
df2.shape

In [ ]:
#Check the null values in each column 
print(df1.isnull().sum())
print(df2.isnull().sum())

In [ ]:
merged_df=df1
merged_df['IMPCH']=df2['IMPCH']
merged_df.head()

In [ ]:
correlation = merged_df['EXPCH'].corr(merged_df['IMPCH'])

# Print the correlation coefficient
print(f"Correlation between the two datasets: {correlation*100}")

In [ ]:
merged_df.plot(figsize=(15,5),legend=True)

In [ ]:
df1.info()
df2.info()

In [ ]:
df1.index
df2.index

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df1 and df2 are your DataFrames containing the covariate variables
# For example, df1['covariate1'] and df2['covariate2'] are the covariate variables

# Plotting covariate variables from df1 and df2 against each other
sns.scatterplot(x=df1['EXPCH'], y=df2['IMPCH'],color='violet')
plt.xlabel('Covariate 1 from EXPCH')
plt.ylabel('Covariate 2 from IMPCH')
plt.title('Scatter Plot of Covariate 1 from EXPCH vs Covariate 2 from IMPCH')
plt.show()

Seasonality Check


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df1['EXPCH'],period=12)  # Assuming monthly data with a seasonal period of 12
plt.figure(figsize=(20, 8))
result.plot()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df2['IMPCH'],period=12)  # Assuming monthly data with a seasonal period of 12
plt.figure(figsize=(18, 8))
result.plot()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
plot_acf(df1['EXPCH'],lags=40,ax=ax);

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
plot_pacf(df1['EXPCH'],lags=25,ax=ax);

ACF and PACF for IMPCH


In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
plot_acf(df2['IMPCH'],lags=40,ax=ax);

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
plot_pacf(df2['IMPCH'],lags=25,ax=ax);

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
result1=adf_test(df1['EXPCH'],title='')

In [ ]:
result2=adf_test(df2['IMPCH'],title='')

As p value for both the Time series is more then 0.05 we can say its not stationary and need differencing.


In [ ]:
from pmdarima import auto_arima

# Assuming 'values' is the column you want to use for time series forecasting
values = merged_df['EXPCH']

# Perform auto_arima on the numerical values
stepwise_fit = auto_arima(values,exogenous=merged_df['IMPCH'],test='adf',suppress_warnings=True,seasonal=True,m=12)

# Print the summary of the fitted model
print(stepwise_fit.summary())

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
train=merged_df.iloc[:421]
test=merged_df.iloc[421:]
print(train.shape,test.shape)

In [ ]:
model = SARIMAX(train['EXPCH'],exog=train['IMPCH'],order=(2,1,2),seasonal_order=(2,0,2,12),enforce_invertibility=True)
results = model.fit()

# Print the model summary
print(results.summary())


In [ ]:
start=len(train)
end=len(train)+len(test)-1
exog_forecast=test['IMPCH']
prediction=results.predict(start=start,end=end,exog=exog_forecast).rename('prediction')

In [ ]:
prediction.head()

In [ ]:
test.head()

In [ ]:
from sklearn.metrics import mean_squared_error
error=np.sqrt(mean_squared_error(test["EXPCH"],prediction))
print(error)

In [ ]:
test['EXPCH'].plot(figsize=(12,8),legend=True)
prediction.plot(legend=True)

In [ ]:
auto_arima(merged_df['IMPCH'],seasonal=True,m=12).summary()

In [ ]:
model_IMPCH = SARIMAX(merged_df['IMPCH'],order=(3,1,0),seasonal_order=(2, 0, [1, 2], 12))
results = model_IMPCH.fit()
IMPCH_PRED = results.predict(len(merged_df),len(merged_df)+47).rename('Forecasted_IMPCH')
print(IMPCH_PRED)

In [ ]:
len(IMPCH_PRED)

In [ ]:
new_df=pd.DataFrame({'IMPCH':IMPCH_PRED,'EXPCH': np.nan})
df_new = pd.concat([merged_df, new_df])
df_new.head()

In [ ]:
df_new.tail()

In [ ]:
len(df_new)

In [ ]:
fcast_start=len(merged_df)
exog_forecast=df_new[421:]['IMPCH']
fcast_val=results.predict(start=fcast_start,end=fcast_start+47,exog=exog_forecast)

In [ ]:
fcast_val

In [ ]:
merged_df.plot(figsize=(12,8),legend=True,color='blue')
IMPCH_PRED.plot(legend=True,color='red')